<a href="https://colab.research.google.com/github/Venkata1345/Mental_health_buddy/blob/main/Mental_health_buddy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install "unsloth[colab-new]"

In [ ]:
pip install "transformers" "peft" "accelerate" "bitsandbytes" "datasets"

In [ ]:
from unsloth import FastLanguageModel
import torch

model_name="unsloth/Qwen3-30B-A3B-bnb-4bit"

model, tokenizer= FastLanguageModel.from_pretrained(model_name=model_name, max_seq_length=2048, dtype=None, load_in_4bit=True)

In [ ]:
# Configure LoRA parameters
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = True,
)

In [ ]:
from datasets import load_dataset

# Define your tokenizer and EOS_TOKEN before this
# tokenizer = ...
EOS_TOKEN = tokenizer.eos_token

# NEW formatting function
def formatting_prompts_func(examples):
    contexts = examples['Context']
    responses = examples['Response']
    texts = []
    for context, response in zip(contexts, responses):
        messages = [
            {"role": "user", "content": context},
            {"role": "assistant", "content": response}
        ]
        formatted_text = tokenizer.apply_chat_template(
            messages, tokenize = False, add_generation_prompt = False
        )
        texts.append(formatted_text + EOS_TOKEN)
    return { "text" : texts }

# Load and map the dataset
dataset = load_dataset("Amod/mental_health_counseling_conversations", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

# Your dataset is now correctly formatted and ready for training.
print("Formatting successful!")

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
import os
os.environ["WANDB_DISABLED"] = "true"

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        max_steps = 60, # Set to a specific number of steps for a quick test run
        num_train_epochs = 3, # Or train for a number of epochs
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 42,
        output_dir = "outputs",
    ),
)

# Start training!
trainer.train()

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# Push just the LoRA adapters (most common)
model.push_to_hub("Abhishek4545/qwen3-30b-mental-health-buddy-lora")
tokenizer.push_to_hub("Abhishek4545/qwen3-30b-mental-health-buddy-lora")

# Or, if you merged the model first:
# merged_model.push_to_hub("your_hf_username/qwen3-8b-mental-health-buddy-merged")
# tokenizer.push_to_hub("your_hf_username/qwen3-8b-mental-health-buddy-merged")